In [1]:
import numpy as np
import cv2
from PIL import Image
import os
import skimage as ski
import matplotlib.pyplot as plt
import pywt
from skimage import io, exposure, color, restoration, filters, feature, util, metrics, data, img_as_float
from natsort import natsorted, ns
from inference import get_model
from inference_sdk import InferenceHTTPClient
from inference.models.utils import get_roboflow_model
from ultralytics import YOLO, settings
from functools import partial
import scipy

SupervisionWarnings: BoundingBoxAnnotator is deprecated: `BoundingBoxAnnotator` is deprecated and has been renamed to `BoxAnnotator`. `BoundingBoxAnnotator` will be removed in supervision-0.26.0.


# **Filter Test Section**

Load training images from dataset.

In [2]:
def load_images(folder):
    images = []
    try:
        for filename in os.listdir(folder):
            img_path = os.path.join(folder, filename)
            try:
                img = Image.open(img_path)
                images.append(img)
            except IOError as e:
                print(f"Could not open image {img_path}: {e}")
    except PermissionError as e:
        print(f"Permission denied: {e}")
    return images

image_path = "C:/Users/needh/Documents/UNI/2024/Sem_2_2024/ICT342/SEAING/Training Images/Underwater Marine Species.v6-marinedataset_v5.yolov8/train/images" # Lachlan desktop path
            #"C:/Users/needh/OneDriveDocuments/Uni/2024/Sem_2_2024/ICT342/SEAING/Training Images/Underwater Marine Species.v6-marinedataset_v5.yolov8/train/images" Lachlan laptop path

images = load_images(image_path)

Apply wavelet-based estimator of the Gaussian noise standard deviation. Used for tuning denoising algorithms with the 'sigma' parameter.

In [ ]:
from skimage.restoration import estimate_sigma

# "C:/Users/needh/OneDrive/Documents/Uni/2024/Sem_2_2024/ICT342/SEAING/Training Images/Underwater Marine Species.v6-marinedataset_v5.yolov8/train/images/0XO5VK05V8L8_jpg.rf.feffd9cb51dd4643f3bbf2174b8adb7d.jpg" Lachlan laptop path

img = cv2.imread("C:/Users/needh/Documents/UNI/2024/Sem_2_2024/ICT342/SEAING/Training Images/Underwater Marine Species.v6-marinedataset_v5.yolov8/train/images/0XO5VK05V8L8_jpg.rf.feffd9cb51dd4643f3bbf2174b8adb7d.jpg")
img = img_as_float(img)

sigma_est = np.mean(estimate_sigma(img, channel_axis=-1))
print(f"Estimated noise standard deviation: {sigma_est}")

Apply gamma correction.

In [ ]:
from skimage import exposure
img = cv2.imread("C:/Users/needh/Documents/UNI/2024/Sem_2_2024/ICT342/SEAING/Training Images/Underwater Marine Species.v6-marinedataset_v5.yolov8/train/images/0XO5VK05V8L8_jpg.rf.feffd9cb51dd4643f3bbf2174b8adb7d.jpg")

gamma_corrected = exposure.adjust_gamma(img, 2.0)

img.mean() > gamma_corrected.mean()
plt.figure(figsize=(12, 8))
plt.subplot(1,2,1)
plt.title("Original Image")
plt.imshow(img[:,:,::-1])

# Gamma Corrected
plt.subplot(1, 2, 2)
plt.title("Gamma Corrected")
plt.imshow(gamma_corrected[:,:,::-1])

plt.show()


Apply local contrast enhancement with Contrast Limited Adaptive Histogram Equalization (CLAHE).

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from skimage import exposure, data, img_as_float

img = cv2.imread("C:/Users/needh/Documents/UNI/2024/Sem_2_2024/ICT342/SEAING/Training Images/Underwater Marine Species.v6-marinedataset_v5.yolov8/train/images/0XO5VK05V8L8_jpg.rf.feffd9cb51dd4643f3bbf2174b8adb7d.jpg")

img = img_as_float(img)

# Apply adaptive histogram equalisation
img_equalised = np.zeros_like(img)
for i in range(3):
    img_equalised[:, :, i] = exposure.equalize_adapthist(img[:, :, i], clip_limit=0.01) # clip_limit is between 0 and 1, higher values increase contrast. 

# Display the original and equalized images
plt.figure(figsize=(12, 8))
plt.subplot(1, 2, 1)
plt.title('Original Image')
plt.imshow(img[:, :, ::-1])

plt.subplot(1, 2, 2)
plt.title('Equalized Image')
plt.imshow(img_equalised[:, :, ::-1])

plt.show()

# Inferior results compared to gamma correction, requires fine-tuning.

Convert sRGB to LAB colour space (Luminance (L), and Chromaticity (A/B)).
Probably not useful but will leave it incase a use is found, apparently can enable easier colour channel manipulation.

In [ ]:
import cv2
import matplotlib.pyplot as plt

img = cv2.imread("C:/Users/needh/Documents/UNI/2024/Sem_2_2024/ICT342/SEAING/Training Images/Underwater Marine Species.v6-marinedataset_v5.yolov8/train/images/0XO5VK05V8L8_jpg.rf.feffd9cb51dd4643f3bbf2174b8adb7d.jpg")

rgb_image = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# Convert from RGB to LAB
lab_image = cv2.cvtColor(rgb_image, cv2.COLOR_RGB2Lab)

plt.imshow(lab_image)
plt.show()

Apply non-local denoising. 

In [ ]:
from skimage.restoration import denoise_nl_means
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage.util import img_as_float

img = cv2.imread("C:/Users/needh/Documents/UNI/2024/Sem_2_2024/ICT342/SEAING/Training Images/Underwater Marine Species.v6-marinedataset_v5.yolov8/train/images/0XO5VK05V8L8_jpg.rf.feffd9cb51dd4643f3bbf2174b8adb7d.jpg")
img = img_as_float(img)

# Apply Non-Local Means denoising
patch_kw = dict(patch_size=5,      # 5x5 patches
                patch_distance=6)  # 13x13 search area

denoised_img = denoise_nl_means(img, h=0.00605, sigma=0.006065078184261036, fast_mode=True, **patch_kw, channel_axis=-1) 
# A high h value may result in a smoother image with an increase in blur.
# For a Gaussian noise of standard deviation sigma, a rule of thumb is to choose the value of h to be sigma of slightly less.

# Display the original and denoised images
plt.figure(figsize=(12, 8))
plt.subplot(1, 2, 1)
plt.title("Original Image")
plt.imshow(img[:, :, ::-1])

plt.subplot(1, 2, 2)
plt.title("Denoised Image")
plt.imshow(denoised_img[:, :, ::-1])

plt.show()

Apply a bilateral denoise filter.

In [ ]:
from skimage.restoration import denoise_bilateral

img = cv2.imread("C:/Users/needh/Documents/UNI/2024/Sem_2_2024/ICT342/SEAING/Training Images/Underwater Marine Species.v6-marinedataset_v5.yolov8/train/images/0XO5VK05V8L8_jpg.rf.feffd9cb51dd4643f3bbf2174b8adb7d.jpg")

# Apply bilateral denoising
bilateral_denoised_img = denoise_bilateral(img, sigma_color=0.05, sigma_spatial=15, bins=10000, channel_axis=-1)

# Display the original and bilateral denoised images
plt.figure(figsize=(12, 8))
plt.subplot(1, 2, 1)
plt.title("Original Image")
plt.imshow(img[:, :, ::-1])

plt.subplot(1, 2, 2)
plt.title("Bilateral Denoised Image")
plt.imshow(bilateral_denoised_img[:, :, ::-1])

plt.show()

Apply unsupervised Wiener-Hunt deconvolution. Does not produce a great result.

In [ ]:
from skimage.restoration import unsupervised_wiener

img = cv2.imread("C:/Users/needh/Documents/UNI/2024/Sem_2_2024/ICT342/SEAING/Training Images/Underwater Marine Species.v6-marinedataset_v5.yolov8/train/images/0XO5VK05V8L8_jpg.rf.feffd9cb51dd4643f3bbf2174b8adb7d.jpg")
img = img_as_float(img)

# Define the Point Spread Function (PSF)
psf = np.ones((5, 5)) / 25

# Apply Unsupervised Wiener deconvolution to each channel separately
deconvolved_img = np.zeros_like(img)
for i in range(3):
    deconvolved_img[:, :, i], _ = unsupervised_wiener(img[:, :, i], psf)

# Display the original and deconvolved images
plt.figure(figsize=(12, 8))
plt.subplot(1, 2, 1)
plt.title("Original Image")
plt.imshow(img[:, :, ::-1])

plt.subplot(1, 2, 2)
plt.title("Unsupervised Wiener Deconvolved Image")
plt.imshow(deconvolved_img[:, :, ::-1])

plt.show()

Apply Richardson-Lucy deconvolution.

In [ ]:
from skimage.restoration import richardson_lucy

img = cv2.imread("C:/Users/needh/Documents/UNI/2024/Sem_2_2024/ICT342/SEAING/Training Images/Underwater Marine Species.v6-marinedataset_v5.yolov8/train/images/0XO5VK05V8L8_jpg.rf.feffd9cb51dd4643f3bbf2174b8adb7d.jpg")
img = img_as_float(img)

psf = np.ones((5, 5)) / 25

deconvolved = np.zeros_like(img)
for i in range(3):  # Apply deconvolution on each channel separately
    deconvolved[:, :, i] = richardson_lucy(img[:, :, i], psf, num_iter=30)

# Display the original and deconvolved images
plt.figure(figsize=(12, 8))
plt.subplot(1, 2, 1)
plt.title("Original Image")
plt.imshow(img[:, :, ::-1])

plt.subplot(1, 2, 2)
plt.title("Deconvolved Image")
plt.imshow(deconvolved[:, :, ::-1])

plt.show()

Apply rescale of intensity levels to improve contrast.

In [ ]:
img = cv2.imread("C:/Users/needh/Documents/UNI/2024/Sem_2_2024/ICT342/SEAING/Training Images/Underwater Marine Species.v6-marinedataset_v5.yolov8/train/images/0XO5VK05V8L8_jpg.rf.feffd9cb51dd4643f3bbf2174b8adb7d.jpg")

rescaled_img = exposure.rescale_intensity(img, in_range='image', out_range=(0, 1)) # Change out_range to adjust exposure.

plt.figure(figsize=(12, 8))
plt.subplot(1, 2, 1)
plt.title("Original Image")
plt.imshow(img[:, :, ::-1])

plt.subplot(1, 2, 2)
plt.title("Rescaled Intensity Image")
plt.imshow(rescaled_img[:, :, ::-1])

plt.show()

Apply histogram equalisation.

In [ ]:
img = cv2.imread("C:/Users/needh/Documents/UNI/2024/Sem_2_2024/ICT342/SEAING/Training Images/Underwater Marine Species.v6-marinedataset_v5.yolov8/train/images/0XO5VK05V8L8_jpg.rf.feffd9cb51dd4643f3bbf2174b8adb7d.jpg")

equalized_img = exposure.equalize_hist(img)

plt.figure(figsize=(12, 8))
plt.subplot(1, 2, 1)
plt.title("Original Image")
plt.imshow(img[:, :, ::-1])

plt.subplot(1, 2, 2)
plt.title("Histogram Equalized Image")
plt.imshow(equalized_img[:, :, ::-1])

plt.show()

Apply Sobel filter for edge detection.

In [ ]:
from skimage.filters import sobel

img = cv2.imread("C:/Users/needh/Documents/UNI/2024/Sem_2_2024/ICT342/SEAING/Training Images/Underwater Marine Species.v6-marinedataset_v5.yolov8/train/images/0XO5VK05V8L8_jpg.rf.feffd9cb51dd4643f3bbf2174b8adb7d.jpg")

# Apply Sobel filter for edge detection
edges = sobel(img)

# Display the original and edge-detected images
plt.figure(figsize=(12, 8))
plt.subplot(1, 2, 1)
plt.title("Original Image")
plt.imshow(img[:, :, ::-1])

plt.subplot(1, 2, 2)
plt.title("Sobel Edge Detection")
plt.imshow(edges, cmap='gray')

plt.show()

Apply Scharr transform to find edges.

In [ ]:
from skimage.filters import scharr

img = cv2.imread("C:/Users/needh/Documents/UNI/2024/Sem_2_2024/ICT342/SEAING/Training Images/Underwater Marine Species.v6-marinedataset_v5.yolov8/train/images/0XO5VK05V8L8_jpg.rf.feffd9cb51dd4643f3bbf2174b8adb7d.jpg")

# Apply Scharr filter for edge detection
scharr_edges = scharr(img)

# Display the original and Scharr edge-detected images
plt.figure(figsize=(12, 8))
plt.subplot(1, 2, 1)
plt.title("Original Image")
plt.imshow(img[:, :, ::-1])

plt.subplot(1, 2, 2)
plt.title("Scharr Edge Detection")
plt.imshow(scharr_edges, cmap='gray')

plt.show()

Use Canny algorithm for edge detection.

In [ ]:
from skimage.feature import canny

img = cv2.imread("C:/Users/needh/Documents/UNI/2024/Sem_2_2024/ICT342/SEAING/Training Images/Underwater Marine Species.v6-marinedataset_v5.yolov8/train/images/0XO5VK05V8L8_jpg.rf.feffd9cb51dd4643f3bbf2174b8adb7d.jpg")

# Convert image to grayscale
gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Apply Canny edge detection
canny_edges = canny(gray_img, sigma=0.006065078184261036)

# Display the original and Canny edge-detected images
plt.figure(figsize=(12, 8))
plt.subplot(1, 2, 1)
plt.title("Original Image")
plt.imshow(img[:, :, ::-1])

plt.subplot(1, 2, 2)
plt.title("Canny Edge Detection")
plt.imshow(canny_edges, cmap='gray')

plt.show()

Apply Simple Linear Iterative Clustering (SLIC) for image segmentation. Not sure how to make this work/be of use. Will need to do more testing/research.

In [ ]:
from skimage.segmentation import slic
from skimage.util import img_as_float

img = cv2.imread("C:/Users/needh/Documents/UNI/2024/Sem_2_2024/ICT342/SEAING/Training Images/Underwater Marine Species.v6-marinedataset_v5.yolov8/train/images/0XO5VK05V8L8_jpg.rf.feffd9cb51dd4643f3bbf2174b8adb7d.jpg")

# Apply SLIC segmentation
segments = slic(img, n_segments=100, compactness=10, sigma=0.006065078184261036, channel_axis=-1, start_label=1)

# Display the segmented image
plt.figure(figsize=(12, 8))
plt.title("SLIC Segmentation")
plt.imshow(segments)

plt.show()

This section is for testing super resolution and Sci-kit image enhancements. Run the above stuff separate from this or change variable names.

Implement OpenCV2 Super Resolution. You have to download each model, this link in useful https://jeanvitor.com/how-use-opencv-superresolution-sr/.
EDSR_x4 provides the best results but is much slower and produces the largest file size.

In [ ]:
import cv2
import argparse
import time
import os
import matplotlib.pyplot as plt

img = cv2.imread("C:/Users/needh/Documents/UNI/2024/Sem_2_2024/ICT342/SEAING/Training Images/Underwater Marine Species.v6-marinedataset_v5.yolov8/train/images/0XO5VK05V8L8_jpg.rf.feffd9cb51dd4643f3bbf2174b8adb7d.jpg")
plt.imshow(img[:,:,::-1])
plt.show()

#Create the SR model
sr = cv2.dnn_superres.DnnSuperResImpl_create()
path = "EDSR_x4.pb" # Replace with name of SR model to use eg. EDSR_x2.pb, ESPCN_x4.pb
sr.readModel(path)
sr.setModel("edsr", 4) # Replace string with name of SR model eg. edsr, espcn, fsrcnn, lapsrn

# Upscale the image
start = time.time()
sr_result = sr.upsample(img)
 
# Resized image
resized = cv2.resize(img,dsize=None,fx=4,fy=4)

plt.figure(figsize=(12, 8))
plt.subplot(1,3,1)

# Original image
plt.title("Original Image")
plt.imshow(img[:,:,::-1])
plt.subplot(1,3,2)

# SR upscaled
plt.title("SR Upscaled")
plt.imshow(sr_result[:,:,::-1])
plt.subplot(1,3,3)

# OpenCV upscaled
plt.title("OpenCV Upscaled")
plt.imshow(resized[:,:,::-1])

plt.show()

Apply Richardson-Lucy deconvolution to the upscaled image.

In [ ]:
import numpy as np
from scipy.signal import convolve2d
from skimage.restoration import richardson_lucy
import matplotlib.pyplot as plt
from skimage import img_as_float

# Convert input image to float
sr_result = img_as_float(sr_result)

# Define the Point Spread Function (PSF) for Richardson-Lucy deconvolution
psf = np.ones((5, 5)) / 25

# Apply Richardson-Lucy deconvolution
deconvolved = np.zeros_like(sr_result)
for i in range(3):  # Apply deconvolution on each channel separately
    deconvolved[:, :, i] = richardson_lucy(sr_result[:, :, i], psf, num_iter=30)

# Display the original and deconvolved images
plt.figure(figsize=(12, 8))
plt.subplot(1, 2, 1)
plt.title("Original Image")
plt.imshow(img[:,:,::-1])

plt.subplot(1, 2, 2)
plt.title("Deconvolved Image")
plt.imshow(deconvolved[:,:,::-1])

plt.show()

# **YOLOv8 Implementation**

Load labels from dataset.

In [2]:
def load_labels(folder):
    labels = {}
    try:
        for filename in os.listdir(folder):
            if filename.endswith(".txt"):
                label_path = os.path.join(folder, filename)
                with open(label_path, 'r') as file:
                    label_data = file.readlines()
                    labels[filename] = [line.strip() for line in label_data]
    except PermissionError as e:
        print(f"Permission denied: {e}")
    return labels

label_path = "C:/Users/needh/Documents/UNI/2024/Sem_2_2024/ICT342/SEAING/Training Images/Underwater Marine Species.v6-marinedataset_v5.yolov8/train/labels"
labels = load_labels(label_path)
print(labels)

{'01A0UCRNRL6O_jpg.rf.2b05ce456cef3f746a5651733c3f6ca3.txt': ['0 0.54609375 0.40546875 0.67734375 0.57109375'], '01A0UCRNRL6O_jpg.rf.59542fe2e9d195f700437df2ee68c0b7.txt': ['0 0.45390625 0.59453125 0.678125 0.571875'], '02VPARJRC8H6_jpg.rf.5f21c606e465fa0c403764e701eeda01.txt': ['0 0.43125 0.48828125 0.2625 0.3984375'], '02VPARJRC8H6_jpg.rf.bd27c94c32d15c1f413ff15378673e44.txt': ['0 0.43125 0.51171875 0.2625 0.3984375'], '02_18_10_18_png.rf.22c792ccf897f114cfa5122dd9dfffe4.txt': ['0 0.7760416671875 0.246296296875 0.7479166671875 0.25740740781250004 0.6802083328124999 0.25648148125 0.6458333328125 0.2629629625 0.5578125 0.29722222187499997 0.4640625 0.321296296875 0.41041666718750003 0.33888888906250003 0.3421875 0.3648148140625 0.2864583328125 0.39351851875 0.2901041671875 0.397222221875 0.32864583281250004 0.39444444375 0.3770833328125 0.3759259265625 0.5880208328125 0.31851851875000003 0.6401041671875001 0.29814814843750004 0.6963541671875 0.2851851859375 0.7494791671875001 0.2824074

Setup/Check Ultralytics and set correct directory path.

In [3]:
print(settings)

value = settings["runs_dir"]
settings.update({"datasets_dir": "C:/Users/needh/Documents/UNI/2024/Sem_2_2024/ICT342/SEAING/Training Images/Underwater Marine Species.v6-marinedataset_v5.yolov8"})

{'settings_version': '0.0.5', 'datasets_dir': 'C:/Users/needh/Documents/UNI/2024/Sem_2_2024/ICT342/SEAING/Training Images/Underwater Marine Species.v6-marinedataset_v5.yolov8', 'weights_dir': 'weights', 'runs_dir': 'runs', 'uuid': '980b41ab137733991b0551b5382cc4ff60b16ed3a0769907367395025ae694f6', 'sync': True, 'api_key': '', 'openai_api_key': '', 'clearml': True, 'comet': True, 'dvc': True, 'hub': True, 'mlflow': True, 'neptune': True, 'raytune': True, 'tensorboard': True, 'wandb': True, 'vscode_msg': True}


Import Roboflow pre-trained dataset "Underwater Marine Species Computer Vision Project"

In [4]:
# Add your own image path to any arbitrary image. It currently only loads one image so we need to figure out how to load all images.

image_list = []

for child in "datasets_dir"
            print(f)



image_file_path = "C:/Users/needh/Documents/UNI/2024/Sem_2_2024/ICT342/SEAING/Training Images/Underwater Marine Species.v6-marinedataset_v5.yolov8/train/images"

# Roboflow model
model_name = "underwater-marine-species"
model_version = "6"

# Get Roboflow face model (this will fetch the model from Roboflow)
model = get_roboflow_model(
    model_id="{}/{}".format(model_name, model_version),

    #Replace ROBOFLOW_API_KEY with your Roboflow API Key
    api_key="J7ljP04frhJRcBuovOmD"
)

In [ ]:
frame = cv2.imread(image_file_path)

results = model.infer(image=frame,
                        confidence=0.5,
                        iou_threshold=0.5)

In [ ]:
if results[0].predictions:
    prediction = results[0].predictions[0]
    print(prediction)
    
    x_center = int(prediction.x)
    y_center = int(prediction.y)
    width = int(prediction.width)
    height = int(prediction.height)

    # Calculate top-left and bottom-right corners from center, width, and height
    x0 = x_center - width // 2
    y0 = y_center - height // 2
    x1 = x_center + width // 2
    y1 = y_center + height // 2
    
    # Currently only takes one input text as a label
    cv2.rectangle(frame, (x0, y0), (x1, y1), (255,255,0), 10)
    cv2.putText(frame, "fish", (x0, y0 - 10), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255, 255, 255), 2)

Display chosen image.

In [8]:
# Show the image from image path. When we get multiple/all images to be loaded we don't want them all to be rendered simultaneously, so this will need to be modified.
cv2.imshow('Image Frame', frame)
cv2.waitKey(0) # waits until a key is pressed
cv2.destroyAllWindows() # destroys the window showing image